In [1]:
import psycopg2
import pandas as pd

In [2]:
DBNAME = "test"
HOST = "35.202.86.236"
USER = "alex"
PASSWORD = "alex"
PORT = "5432"

conn = psycopg2.connect(dbname=DBNAME, user=USER, password=PASSWORD, host=HOST, port=PORT)


In [3]:
cursor = conn.cursor()

In [4]:
USER = 'cac93e86-cad0-4030-839b-0415300528ab' # alex
# USER = '9b8e372a-b5d4-4771-8741-511bae5d9193'

records = []
try: 
    cursor.execute("""
    SELECT
        f.user_uuid,
        log_id,
        protein,
        carbohydrates,
        fats,
        calories,
        servings,
        log_date,
        LOWER(TRIM(food_name)) as food_name,
        serving_size,
        group_id,
        CASE
            WHEN EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) >= 0 AND EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) < 6 THEN 'Night'
            WHEN EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) >= 6 AND EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) < 12 THEN 'Morning'
            WHEN EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) >= 12 AND EXTRACT(HOUR FROM log_date AT TIME ZONE 'UTC' AT TIME ZONE u.timezone) < 18 THEN 'Afternoon'
            ELSE 'Evening'
        END AS time_of_day
    FROM
        food_log f
    JOIN 
        users u
    ON
        f.user_uuid = u.user_uuid
    WHERE
        f.user_uuid = '{}'
    GROUP BY 
        f.user_uuid,
        log_id,
        protein,
        carbohydrates,
        fats,
        calories,
        servings,
        log_date,
        food_name,
        serving_size,
        group_id,
        time_of_day
    ORDER BY
        log_date DESC

    """.format(USER))
    records = cursor.fetchall()
    # Create a DataFrame from the fetched records
    df = pd.DataFrame(records, columns=[
        'user_uuid',
        'log_id',
        'protein',
        'carbohydrates',
        'fats',
        'calories',
        'servings',
        'log_date',
        # 'food_name_clean',
        'food_name',
        # 'image_path',
        'serving_size',
        'group_id',
        'time_of_day'

    ])


    cursor.close()
    conn.close()

    # # Save to file
    df.to_csv('./data/user.csv', index=False)
except Exception as e:
    print(e)
    conn.rollback()


In [5]:
user = pd.read_csv('./data/user.csv')

userData = user[['food_name', 'log_date', 'time_of_day']].copy()  # Make a copy of the DataFrame

# Extract weekday from log_date and assign it to a new column 'weekday' using .loc
userData.loc[:, 'weekday'] = pd.to_datetime(userData['log_date']).dt.day_name()

# userData.loc[:, 'time'] = pd.to_datetime(userData['log_date']).dt.time

userData.drop(['log_date'], axis=1, inplace=True)  # Drop the 'log_date' column

userData['food_name'] = userData['food_name'].str.lower()  # Capitalize the food names


userData.to_csv('./data/userData.csv', index=False)  # Save to file

userData


,food_name,time_of_day,weekday
0,veggie patty,Evening,Wednesday
1,salad,Evening,Wednesday
2,chicken with mole,Evening,Wednesday
3,camomile tea,Afternoon,Tuesday
4,quesadilla,Afternoon,Tuesday
5,scrambled eggs,Afternoon,Monday
6,cheese ziti pasta,Afternoon,Sunday
7,omelette,Afternoon,Saturday
8,cereal,Afternoon,Thursday
9,scrambled eggs,Afternoon,Thursday
